In [7]:
import cv2
import tensorflow as tf
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys
from tensorflow.contrib import rnn

In [8]:
############### Global Parameters ###############
# path
train_path = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/training/'
test_path = '/media/user/Hard_Disk/Dataset/child_accident_2/batch_start2end_score/testing/'
# demo_path = './dataset/features/testing/'
# default_model_path = './model/demo_model'
save_path = './model2/'
# video_path = './dataset/videos/training/positive/'
# batch_number
train_num = 101
test_num = 20

In [9]:
class ModelRNN:
    def __init__(self, n_classes, rnn_size, max_seq_sz, num_layers):
        self.input_seq = tf.placeholder('float', [None, max_seq_sz, n_classes])
        self.target = tf.placeholder('float', [None, n_classes])
        self.nClasses = n_classes
        self.rnn_size = rnn_size
        self.max_seq_sz = max_seq_sz
        self.num_layers = num_layers
        
        self.__build()

    def __weight_variable(self, shape, myName):
        initial = tf.random_normal(shape, stddev=0.1, name=myName)
        return tf.Variable(initial)
    
    def __bias_variable(self, shape, myName):
        initial = tf.constant(0.1, shape=shape, name=myName)
        return tf.Variable(initial)
        
    def __build(self):
        w_fc_in = self.__weight_variable([self.nClasses, 128], 'w_fc_in')
        b_fc_in = self.__bias_variable([128], 'b_fc_in')
        
        w_fc_o = self.__weight_variable([self.rnn_size, 128], 'w_fc_o')
        b_fc_o = self.__bias_variable([128], 'b_fc_o')
                
        w_output_action = self.__weight_variable([128, self.nClasses], 'w_fc_in')
        b_output_action = self.__bias_variable([self.nClasses], 'b_fc_in')
        
        w_output_len = self.__weight_variable([128, 2], 'w_fc_in')
        b_output_len = self.__bias_variable([2], 'b_fc_in')
        
        x = tf.reshape(self.input_seq, [-1, self.nClasses])
        h1 = tf.nn.relu(tf.matmul(x, w_fc_in) + b_fc_in)
        h1 = tf.reshape(h1, [-1,self.max_seq_sz,128])
        #rnn
        h1 = tf.unstack(h1, axis=1)
        def get_cell():
            return rnn.GRUCell(self.rnn_size)   
        gru_cell = rnn.MultiRNNCell([get_cell() for _ in range(self.num_layers)])
        outputs, states = rnn.static_rnn(gru_cell, h1, dtype=tf.float32) 
        #fc_o
        h2 = tf.nn.relu(tf.matmul(outputs[-1], w_fc_o) + b_fc_o)
        #output
        output_label = tf.matmul(h2, w_output_action) + b_output_action
        output_len = tf.nn.relu(tf.matmul(h2, w_output_len) + b_output_len)
        #    
        self.prediction = tf.concat([output_label, output_len], 1)
        self.saver = tf.train.Saver(write_version=tf.train.SaverDef.V2, max_to_keep=100)
    
    def train(self, sess, model_save_path, batch_gen, nEpochs, save_freq, batch_size):
        gt_labels = self.target[:,:-2]
        gt_length = self.target[:,-2:]
        predicted_labels = self.prediction[:,:-2]
        predicted_length = self.prediction[:,-2:]
        
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=gt_labels, logits=predicted_labels, dim=1))
        loss += tf.reduce_mean(tf.square( gt_length - predicted_length ))
        
        optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
                
        sess.run(tf.global_variables_initializer())
      
        for epoch in range(nEpochs):
            epoch_loss = 0
            i=0
            while(batch_gen.has_next()):
                batch_in, batch_target = batch_gen.next_batch(batch_size)
                _, err = sess.run([optimizer, loss], feed_dict={self.input_seq: batch_in, self.target: batch_target})
                i=i+1
                epoch_loss += err       
            batch_gen.reset()
            
            if epoch%save_freq==0:  
                print ('Epoch', (epoch+1), 'completed out of',nEpochs,'epoch loss: %.2f'%(epoch_loss/i))
                path = model_save_path+"/epoch-"+str(epoch+1)
                if not os.path.exists(path):
                    os.makedirs(path)
                self.saver.save(sess, path+"/model.ckpt")    
    
    

    def predict(self, sess, model_save_path, pred_len, label_seq, length_seq, actions_dict, T):
        self.saver.restore(sess, model_save_path)
        l = 0
        while l < pred_len:
            p_seq = np.zeros((self.max_seq_sz, self.nClasses+1))
            for i in range(len(label_seq[-self.max_seq_sz:])):
                p_seq[i][-1] = length_seq[i]/T
                p_seq[i][ actions_dict[label_seq[i]] ] = 1

            result = self.prediction.eval({self.input_seq:[p_seq]})[0]

            if int(result[-1]*T) > 0:
                length_seq[-1] += result[-1]*T
                l = l + int(result[-1]*T)
            if int(result[-2]*T) > 0:
                l = l + int(result[-2]*T)
                label_seq.append(actions_dict.keys()[actions_dict.values().index(np.argmax(result[:-2]))])
                length_seq.append(result[-2]*T)
            if int(result[-1]*T) == 0 and int(result[-2]*T) == 0:
                l = l+pred_len
                length_seq[-1] += pred_len

        return label_seq, length_seq

In [12]:
class Base_batch_generator(object):
    
    def __init__(self):
        self.list_of_examples = []
        self.index = 0
    
    def number_of_examples(self):
        return len(self.list_of_examples)
        
    def reset(self):
        self.index = 0
        return
        
    def has_next(self):
        if self.index < len(self.list_of_examples):
            return True
        return False
    
    def read_data(self, list_of_videos):
        raise NotImplementedError()
    
    def next_batch(self, batch_size):
        raise NotImplementedError()

In [13]:
class RNN_batch_generator(Base_batch_generator):
    
    def __init__(self, nClasses, n_iterations, max_seq_sz, actions_dict, alpha):
        super(RNN_batch_generator, self).__init__()
        self.n_iterations = n_iterations
        self.nClasses = nClasses
        self.max_seq_sz = max_seq_sz
        self.actions_dict = actions_dict
        self.alpha = alpha


    def read_data(self, list_of_videos):
        
        for vid in list_of_videos:
                
            file_ptr = open(vid, 'r')
            content = file_ptr.read().split('\n')[:-1]
            
            label_seq, length_seq = get_label_length_seq(content) 
            T = (1.0/self.alpha)*len(content)
        
            for itr in range(self.n_iterations):
                #list of partial length of each label in the sequence
                rand_cuts = []
                for i in range(len(label_seq)-1):
                    rand_cuts.append( int( length_seq[i] * float(itr+.5)/self.n_iterations  ) )
                    
                for i in range(len(rand_cuts)):
                    seq_len = i+1
                    p_seq = []
                    for j in range(seq_len):
                        p_seq.append(np.zeros((self.nClasses+1)))
                        if j == seq_len-1:
                            p_seq[-1][-1] = rand_cuts[j]/T
                        else:
                            p_seq[-1][-1] = length_seq[j]/T
                        p_seq[-1][self.actions_dict[label_seq[j]]] = 1
                        
                    for j in range(self.max_seq_sz - seq_len):
                        p_seq.append(np.zeros((self.nClasses+1)))
                    
                    p_tar = np.zeros((self.nClasses+2))
                    #target length
                    if i != len(rand_cuts)-1:
                        p_tar[-2] = rand_cuts[i+1]/T
                    else:
                        p_tar[-2] = length_seq[i+1]/T
                    #remaining length
                    p_tar[-1] = (length_seq[i]-rand_cuts[i])/T
                    #target action
                    p_tar[ self.actions_dict[label_seq[i+1]] ] = 1
                    
                    example = [p_seq, p_tar, seq_len]
                    self.list_of_examples.append(example)
                    
        random.shuffle(self.list_of_examples) 
        return


    def next_batch(self, batch_size):
        batch = np.array( sorted(self.list_of_examples[self.index:self.index+batch_size], key=lambda x: x[2], reverse=True) )
        self.index += batch_size
        batch_vid = list(batch[:,0])
        batch_target = list(batch[:,1])
                
        return batch_vid, batch_target

In [10]:
model = ModelRNN(2, 512, 100, 2)